# Fukuhara-Seminar-Week-01: Python Analysis Reviewing Textbook

## 0.0: Contents Table
- Data Colecting
  - データの概要とか、期間や種類とか、選んだ理由ちょいと
- Regression & Hypothesis Testing
  - CAPMのベータ計算：
  - 重回帰分析：
- Asuumption
  - (Zero mean)
  - Homoscedasticy
    - breusc-pagan
  - No Autocorrelation(DW)
  - Exogeneity(dollow from non-stochastic X)...N/A out of scope
    - Cov(u, X)=0
  - Normality→python guide
    - jarque bera
  - Linearity
    - RESET
  - Stability
    - cusum
- Minimum Variance Portofolio and Market Portofolio
  - Portfolio theory
- (Copula)


## 1.0: Make monthly stock dataset for 10 years

In [37]:
import pandas as pd
from pandas_datareader import data as web
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import yfinance as yf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from scipy.optimize import minimize
import cvxpy as cp

# ターゲットとなる企業のティッカーと企業名のリスト
tickers = [
    "1925", "1928", "2413", "2502", "2503", "2802", "2914", "3382", "3402", "3407", "4063", "4188", "4452", "4502", "4503", "4507", "4519", "4523", "4528", "4543", "4568", "4578", "4661", "4689", "4901", "4911", "5020", "5108", "5401", "5713", "5802", "6098", "6178", "6273", "6301", "6326", "6367", "6501", "6502", "6503", "6586", "6594", "6645", "6702", "6752", "6758", "6861", "6869", "6902", "6920", "6954", "6971", "6981", "7011", "7201", "7203", "7267", "7269", "7270", "7309", "7733", "7741", "7751", "7832", "7974", "8001", "8002", "8031", "8035", "8053", "8058", "8113", "8267", "8306", "8308", "8309", "8316", "8411", "8591", "8604", "8630", "8697", "8725", "8750", "8766", "8801", "8802", "8830", "9020", "9021", "9022", "9101", "9202", "9432", "9433", "9434", "9735", "9843", "9983", "9984"
]

companies = [
    '大和ハウス工業','積水ハウス','エムスリー','アサヒグループホールディングス','キリンホールディングス','味の素','日本たばこ産業','セブン＆アイ・ホールディングス','東レ','旭化成','信越化学工業','三菱ケミカルホールディングス','花王','武田薬品工業','アステラス製薬','塩野義製薬','中外製薬','エーザイ','小野薬品工業','テルモ','第一三共','大塚ホールディングス','オリエンタルランド','Ｚホールディングス','富士フイルムホールディングス','資生堂','JXTGホールディングス','ブリヂストン','日本製鉄','住友金属鉱山','住友電気工業','リクルートホールディングス','日本郵政','SMC','小松製作所','クボタ','ダイキン工業','日立製作所','東芝','三菱電機','マキタ','日本電産','オムロン','富士通','パナソニック','ソニー','キーエンス','シスメックス','デンソー','レーザーテック','銘柄名','ファナック','京セラ','村田製作所','三菱重工業','日産自動車','トヨタ自動車','本田技研工業','スズキ','SUBARU','シマノ','オリンパス','HOYA','キヤノン','バンダイナムコホールディングス','任天堂','伊藤忠商事','丸紅','三井物産','東京エレクトロン','住友商事','三菱商事','ユニ・チャーム','イオン','三菱UFJフィナンシャル・グループ','りそなホールディングス','三井住友トラスト・ホールディングス','三井住友フィナンシャルグループ','みずほフィナンシャルグループ','オリックス','野村ホールディングス','SOMPOホールディングス','日本取引所グループ','MS＆ADインシュアランスグループホールディングス','第一生命ホールディングス','東京海上ホールディングス','三井不動産','三菱地所','住友不動産','東日本旅客鉄道','西日本旅客鉄道','東海旅客鉄道','日本郵船','ANAホールディングス','日本電信電話','KDDI','ソフトバンク','セコム','ニトリホールディングス','ファーストリテイリング','ソフトバンクグループ'
]

In [ ]:
# 10年前の日付を取得
end = datetime.now()
start = end - timedelta(days=10*365)

# データを格納するための空のデータフレームを作成
df_all = pd.DataFrame()

# 各ティッカーに対してデータを取得
for i, ticker in enumerate(tickers):
    try:
        # データを取得（Yahoo Finance）
        df = web.get_data_yahoo(f"{ticker}.T", start, end)
        # 日ごとの平均株価を計算
        df['Mean'] = (df['High'] + df['Low']) / 2
        # 企業名を追加
        df['Name'] = companies[i]
        # 月ごとの平均株価を計算
        df_resampled = df.resample('M').mean()[['Mean']]
        df_resampled['Name'] = companies[i]
        # df_allと連結
        df_all = pd.concat([df_all, df_resampled[['Name', 'Mean']]])
    except Exception as e:
        print(f"Error with ticker {ticker}: {str(e)}")
        # エラーが発生した場合、日付範囲内でゼロを持つ行を作成し、df_allに追加します
        date_range = pd.date_range(start=start, end=end, freq='M')
        df_zero = pd.DataFrame({'Mean': 0}, index=date_range)
        df_zero['Name'] = companies[i]
        df_all = pd.concat([df_all, df_zero[['Name', 'Mean']]])

# ピボット処理を適用し、'Date'をインデックスに、'Name'をカラム名に、'Mean'を値にします。
df_pivot = df_all.pivot_table(values='Mean', columns='Name', index=df_all.index)

# CSVに出力
df_pivot.to_csv("monthly_stock_prices.csv")

In [32]:
df = pd.read_csv("./monthly_stock_prices.csv")
df

,Date,ANAホールディングス,HOYA,JXTGホールディングス,KDDI,MS＆ADインシュアランスグループホールディングス,SMC,SOMPOホールディングス,SUBARU,みずほフィナンシャルグループ,...,武田薬品工業,積水ハウス,第一三共,第一生命ホールディングス,花王,西日本旅客鉄道,資生堂,野村ホールディングス,銘柄名,Ｚホールディングス
0,2013-10-31,51.934722,3126.666667,487.583333,NaN,1355.000000,22778.611111,1098.402778,8539.722222,1623.027778,...,4647.222222,1313.777778,590.740741,1044.638889,3083.805556,2568.722222,1726.000000,2475.722222,16254.444444,506.388889
1,2013-11-30,51.781250,3205.750000,517.575000,NaN,1484.850000,23238.250000,1278.937500,8773.000000,1745.025000,...,4788.250000,1374.475000,624.091669,1103.874994,3320.875000,2499.400000,1721.400000,2671.425000,16329.250000,478.975000
2,2013-12-31,53.545000,3314.000000,525.675000,NaN,1635.450000,24987.000000,1441.362500,8936.750000,1800.050000,...,5034.000000,1423.575000,624.408331,1128.500003,3296.375000,2437.050000,1681.175000,2835.950000,17654.000000,551.475000
3,2014-01-31,56.416316,3194.105263,528.184211,NaN,1700.210526,27258.552632,1375.315789,9248.157895,1723.078947,...,4816.157895,1497.184211,622.921048,1092.201745,3308.184211,2391.631579,1634.421053,2880.631579,17629.210526,615.210526
4,2014-02-28,55.782894,3053.552632,505.473684,NaN,1520.500000,25048.289474,1228.144737,8684.210526,1528.473684,...,4775.000000,1344.973684,563.570176,1015.499998,3304.447368,2273.236842,1691.526316,2634.578947,16929.342105,607.578947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2023-06-30,165.091363,3692.863636,483.577271,1520.920455,2618.681818,80115.000000,2472.500000,23243.181818,2511.250000,...,4532.295455,2789.045455,4760.454545,3273.840909,5164.181818,3532.000000,6653.159091,6306.704545,5071.000000,354.400002
117,2023-07-31,164.922500,3743.225000,500.627500,1542.887500,2771.737500,76493.500000,2391.075000,23740.875000,2611.137500,...,4395.800000,2896.175000,4067.825000,3230.000000,5302.475000,3534.225000,6303.025000,6440.800000,4782.350000,382.859999
118,2023-08-31,163.245456,3595.795455,525.825000,1615.363636,2689.909091,69580.681818,2544.454545,21467.727273,2624.647727,...,4423.227273,2891.272727,4266.295455,3129.625000,5602.204545,3619.636364,6116.227273,6147.545455,4118.954545,412.686365
119,2023-09-30,175.507499,3625.950000,597.472498,1728.875000,3012.650000,69378.000000,2653.512500,20799.125000,2864.050000,...,4670.750000,3054.837500,4123.475000,3483.875000,5582.150000,3800.550000,5631.700000,6650.050000,4044.475000,431.795002
